In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.constants import R
import pandas as pd
import numpy as np
import os
import glob 

import ipywidgets as widgets

import pygaps as pg
from bet_class import BET

# BET analysis dashboard - BelMiniX

Written in Python using [pyGAPS](https://pygaps.readthedocs.io/en/master/)
, [Voila](https://voila.readthedocs.io/en/stable/using.html) and JupyterLab.  

Source code available on: [https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis](https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis)

by,  
Nicolai Haaber Junge  
Heterogeneous Catalysis,  
Centre for materials science and nanotechnology  
University of Oslo  
11.07.2021

### This dashboard performs BET analysis on BelMiniX .DAT measurment files.

The BET analysis uses the Rouquerol rules for selecting a pressure range. Note,
that you can also set a manual pressure range if you wish.

With this dashboard you can:
- Do BET analysis in single and batch mode
- Quickly batch process a folder of .DAT files
- Export results as individual or multiple files in excel and csv format.

---

In [ ]:
# INPUT PAGE

lbl = widgets.Label('Select the folder "above" your measurment folder', layout=widgets.Layout(grid_area='lbl', justify_self='start'))
main_folder_lbl = widgets.Label('Main folder', grid_area='main_f_lbl')
main_folder = widgets.Text(layout=widgets.Layout(width='95%', height='30px', grid_area='main_f'))  #  Main folder select

select_folder_lbl = widgets.Label('Selected folder', layout=widgets.Layout(grid_area='sel_f_lbl'))
select_folder = widgets.Dropdown(layout=widgets.Layout(width='95%', height='30px', grid_area='sel_f'))

file_lbl = widgets.Label('File select', layout=widgets.Layout(grid_area='file_lbl'))
file_folder = widgets.SelectMultiple(layout=widgets.Layout(width='95%', height='200px', grid_area='file_f'))

def on_value_change(change):
    folder = os.listdir(change.new)
    select_folder.options=folder

def on_value_change_2(change):
    fold = os.path.join(main_folder.value, change.new)
    files = [_ for _ in os.listdir(fold) if _.endswith('.DAT')]
    file_folder.options = files
    run_analysis.disabled = False
    
main_folder.observe(on_value_change, names='value')
select_folder.observe(on_value_change_2, names='value')  


input_options = widgets.GridBox(
        children=[lbl, main_folder_lbl, main_folder, 
                  select_folder_lbl, select_folder, 
                  file_lbl, file_folder],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "lbl lbl lbl lbl"
            "main_f_lbl main_f_lbl main_f_lbl main_f_lbl"
            "main_f main_f main_f main_f"
            "sel_f_lbl sel_f_lbl sel_f_lbl sel_f_lbl"
            "sel_f sel_f sel_f sel_f"
            "file_lbl file_lbl file_lbl file_lbl"
            "file_f file_f file_f file_f"
            ''')
       )



In [ ]:
# EXPORT TAB

def create_toggle(descr, area_name):
    return widgets.ToggleButton(description=descr,
                 layout=widgets.Layout(width='auto', grid_area=area_name, justify_self='start'))

check1 = create_toggle('Export folder', 'check1')
check2 = create_toggle('Export selected', 'check2')
check3 = create_toggle('Save as individual files', 'check3')
check4 = create_toggle('To excel', 'check4')
check5 = create_toggle('To csv', 'check5')

label1 = widgets.Label('Batch export options', layout=widgets.Layout(width='auto', grid_area='label1'))
label2 = widgets.Label('File options', layout=widgets.Layout(width='auto', grid_area='label2'))


note = 'It is recommended to set a manual BET \npressure range when batch exporting.\n\n'\
        'Run a single BET calculation first to \nget an idea of a good pressure range.'

label_opt = widgets.Textarea(note, disabled=True, layout=widgets.Layout(width='auto', height='90px', grid_area='label_opt'))


export_options = widgets.GridBox(children=[check1, check2, check3, check4, check5, label1, label2, label_opt],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto 100px auto 50px',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "label1 label1 label1 label1"
            "check1 check1 check2 check2"
            "label_opt label_opt label_opt label_opt"
            "label2 label2 label2 label2"
            "check3 check3 check4 check5"
            ''')
       )

In [ ]:
# INPUT AND EXPORT TABS

select_fol = widgets.VBox([select_folder, file_folder], layout=widgets.Layout(width='100%'))

tab_contents = ['Data Input', 'Batch & Export']
children = [input_options, export_options]
tab = widgets.Tab(layout=widgets.Layout(width='99%', height='600px', grid_area='tab'))
tab.children = children
it = [tab.set_title(i, title) for i, title in enumerate(tab_contents)]

In [ ]:
# BET PAGE

######## OUTPUT WIDGETS
isotherm_out = widgets.Output()
bet_plot = widgets.Output()
roq_plot = widgets.Output()
bet_res = widgets.Output()

plot_outputs = [isotherm_out, bet_plot, roq_plot, bet_res]

### BUTTONS ETC
run_analysis = widgets.Button(description='Calc. BET', disabled=True, layout=widgets.Layout(grid_area='run_anal'))
manual_press_check = widgets.ToggleButton(False, description='Set pressure range', layout=widgets.Layout(grid_area='man_pres_check'))

press_range_lbl = widgets.Label('Manual press. range', layout=widgets.Layout(grid_area='press_range_lbl'))
press_range_slider = widgets.FloatRangeSlider(
    value=[1e-3, 1e-1],
    min=0,
    max=0.1,
    step=1e-3,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    layout=widgets.Layout(width='auto', grid_area='pres_range')
)


material  = widgets.Label('Material', layout=widgets.Layout(width='auto', grid_area='material'))
material_name = widgets.Text(layout=widgets.Layout(width='auto', grid_area='mat_name'))
meas_id = widgets.Label('Measurmet ID.', layout=widgets.Layout(width='auto', grid_area='meas_id'))
meas_id_name  = widgets.Text(layout=widgets.Layout(width='auto', grid_area='meas_id_name'))


##### EVENTS
def run_bet(b):
    
    for output in plot_outputs:
        output.clear_output()
    
    file_select = os.path.join(main_folder.value, select_folder.value, file_folder.value[0])
    bet = BET(file_select)
    with bet_res:
        res = bet.BET_results 
        df = pd.DataFrame(res.values(), columns=['value'])
        df = df.set_index(pd.Index([*res]))
        df['units'] = ['m^2/g', '', '', 'mol/g' , '', '', '', '', '', '', '']
        display(df)
        
    with isotherm_out:
        bet.plot_isotherm()
    
    with bet_plot:
        bet.plot_bet()

    with roq_plot:
        bet.plot_roq()
        
        
run_analysis.on_click(run_bet)

### LAYOUT
BET_options = widgets.GridBox(
    children=[run_analysis, manual_press_check, press_range_slider, press_range_lbl
             , material, material_name, meas_id, meas_id_name],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='60px 30px 40px auto auto auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            ". run_anal run_anal ."
            "material material material material"
            "mat_name mat_name mat_name mat_name"
            "meas_id meas_id meas_id meas_id"
            "meas_id_name meas_id_name meas_id_name meas_id_name"
            "man_pres_check . . ."
            "press_range_lbl press_range_lbl press_range_lbl press_range_lbl"
            "pres_range pres_range pres_range pres_range"
            ''')
       )


tab_BET_contents = ['BET']
BET_children = [BET_options]
tab_BET = widgets.Tab(layout=widgets.Layout(width='99%', height='600px', grid_area='bet_tab'))
tab_BET.children = BET_children
bet_titles = [tab_BET.set_title(i, title) for i, title in enumerate(tab_BET_contents)]

In [ ]:
## PLOTTING

tab_plot_contents = ['Isotherm', 'BET plot', 'Rouquerol plot']
plot_children = [isotherm_out, bet_plot, roq_plot]
tab_plot = widgets.Tab(layout=widgets.Layout(width='95%', height='600px', grid_area='plot_tab'))
tab_plot.children = plot_children
plot_titles = [tab_plot.set_title(i, title) for i, title in enumerate(tab_plot_contents)]

In [ ]:
# MAIN WINDOW

main = widgets.GridBox(children=[tab, tab_BET, tab_plot],
        layout=widgets.Layout(
            justify_content='center',
            width='97%',
            grid_template_rows='auto',
            grid_template_columns='25% 22% 50%',
            grid_template_areas='''
            "tab bet_tab plot_tab"
            ''')
       )

display(main)